In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [8]:
IMAGE_SIZE = 200

In [5]:


labels = []
#read text file
with open('labels.txt') as f:
    labels = f.readlines()
    labels = [x.strip() for x in labels]


In [18]:
original_model = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [19]:
original_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [20]:
# convert to tf lite
converter = tf.lite.TFLiteConverter.from_keras_model(original_model)
tflite_model = converter.convert()
#save the model
open("converted_model.tflite", "wb").write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\oskar\AppData\Local\Temp\tmpgd64hclt\assets


INFO:tensorflow:Assets written to: C:\Users\oskar\AppData\Local\Temp\tmpgd64hclt\assets


91369340

In [33]:
# run the model in inference mode
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()


# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(output_details)

# read an image and resize it
img = tf.keras.preprocessing.image.load_img(r"C:\Users\oskar\OneDrive - Jonkoping University\Dokument\SOTA\imagenet-mini\val\n02102177\ILSVRC2012_val_00016315.JPEG", target_size=(299, 299))

img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array

interpreter.set_tensor(input_details[0]['index'], [img_array])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
# get the top 5 predictions
top_5 = np.argsort(output_data[0])[-5:]
top_5
# replace the labels with labels from the text file
for i in top_5:
    print(labels[i])

[{'name': 'StatefulPartitionedCall:0', 'index': 227, 'shape': array([   1, 1000]), 'shape_signature': array([  -1, 1000]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
clumber, clumber spaniel,
Brittany spaniel,
English springer, English springer spaniel,
cocker spaniel, English cocker spaniel, cocker,
Welsh springer spaniel,


In [41]:

#  test the inference time
import time
start_time = time.time()
interpreter.invoke()
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.5196387767791748 seconds ---
